In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, TensorDataset
from torch.amp import GradScaler, autocast
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import sys
import os

from ResNext_CBAM.CBAM_conv import CBAM
from ResNext_CBAM.ResNext_bottleneck import Resnext_bottleneck_block, Resnext_bottleneck_transpose_block

def rao_split(rao_amp_df, rao_amp_iso_df):
    rao_res = rao_amp_df[:, :, :10] - rao_amp_iso_df
    rao_iso = rao_amp_iso_df
    return rao_res, rao_iso

rao_res, rao_iso = rao_split(rao_amp_df, rao_amp_iso_df)

X_train, X_test, iso_train, iso_test, res_train, res_test = train_test_split(
    features_df,
    rao_iso,
    rao_res,
    test_size=0.3,
    random_state=42
)


from sklearn.preprocessing import MinMaxScaler
scaler_in = MinMaxScaler()
scaler_in.fit(X_train)
X_train_scaled = scaler_in.transform(X_train)
X_test_scaled = scaler_in.transform(X_test)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

X_train_t = torch.tensor(X_train_scaled, dtype=torch.float32, device=device)
X_test_t = torch.tensor(X_test_scaled, dtype=torch.float32, device=device)
iso_train_t = torch.tensor(iso_train, dtype=torch.float32, device=device)
iso_test_t = torch.tensor(iso_test, dtype=torch.float32, device=device)
res_train_t = torch.tensor(res_train, dtype=torch.float32, device=device)
res_test_t = torch.tensor(res_test, dtype=torch.float32, device=device)

train_dataset = TensorDataset(X_train_t, iso_train_t, res_train_t)
test_dataset = TensorDataset(X_test_t, iso_test_t, res_test_t)

def set_seed(seed_value):
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed(seed_value)
    np.random.seed(seed_value)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

class Stage1Model(nn.Module):
    def __init__(self, decoder_in, out_size, conv_block, convt_block, n_blocks,
                 decoder_channel_shrink=1, width_ratio=1, use_cbam=(True, False, True, False),
                 CBAM=None, CBAM_skip=False):
        super(Stage1Model, self).__init__()
        self.in_channels, self.in_height, self.in_width = decoder_in
        self.out_height, self.out_width = out_size
        self.width_ratio = width_ratio
        self.CBAM = CBAM

        self.in_channels_forward = self.in_channels
        self.channel_shrink_rate = decoder_channel_shrink
        self.encoder = nn.Sequential(
            nn.Linear(7, self.in_channels),
            nn.ReLU(True),
        )
        self.decoder_fc = nn.Sequential(
            nn.Linear(self.in_channels, self.in_channels * self.in_height * self.in_width),
            nn.ReLU(True)
        )

        # Decoder layers
        self.layer1 = self._make_layer('layer1', conv_block, convt_block,
                                       self.in_channels_forward // self.channel_shrink_rate,
                                       n_blocks[0], stride=2, shrink=self.width_ratio,
                                       cardinality=32, convt_padd=(2, 1),
                                       use_cbam=use_cbam[0], CBAM_skip=CBAM_skip)
        self.layer2 = self._make_layer('layer2', conv_block, convt_block,
                                       self.in_channels_forward // (self.channel_shrink_rate * 2),
                                       n_blocks[1], stride=2, shrink=self.width_ratio,
                                       cardinality=32, use_cbam=use_cbam[1],
                                       CBAM_skip=CBAM_skip)
        self.layer3 = self._make_layer('layer3', conv_block, convt_block,
                                       self.in_channels_forward // (self.channel_shrink_rate * 4),
                                       n_blocks[2], stride=2, shrink=self.width_ratio,
                                       cardinality=32, convt_padd=(1, 3),
                                       convt_out_padd=(1, 0),
                                       use_cbam=use_cbam[2], CBAM_skip=CBAM_skip)
        self.layer4 = self._make_layer('layer4', conv_block, convt_block,
                                       1,
                                       n_blocks[3], stride=2, shrink=self.width_ratio,
                                       cardinality=1, convt_out_padd=(1, 1),
                                       use_cbam=use_cbam[3], CBAM_skip=CBAM_skip)

        self.iso_conv = nn.Conv2d(1, 1, kernel_size=1, stride=1, padding=0)
        self.final_fc=nn.Linear(360,360)

        # Initialization
        self._initialize_weights()

    def _make_layer(self, name, conv_block, convt_block, out_channels, n_block, stride, shrink,
                    convt_padd=(1, 1), convt_out_padd=(0, 0), cardinality=32,
                    reduc_ratio_CBAM=16, use_cbam=True, CBAM_skip=False):
        layers = nn.Sequential()
        for block_idx in range(n_block):
            name_ = f'{name}_block_{block_idx}'
            if block_idx == 0:
                layers.add_module(name_, convt_block(
                    self.in_channels, out_channels, stride=stride,
                    cardinality=cardinality, shrink=shrink,
                    convt_padd=convt_padd, convt_out_padd=convt_out_padd,
                    reduc_ratio_CBAM=reduc_ratio_CBAM,
                    use_cbam=use_cbam,
                    CBAM=self.CBAM,
                    CBAM_skip=CBAM_skip))
                self.in_channels = out_channels
            else:
                layers.add_module(name_, conv_block(
                    self.in_channels, out_channels, stride=1,
                    cardinality=cardinality, shrink=shrink,
                    reduc_ratio_CBAM=reduc_ratio_CBAM,
                    use_cbam=use_cbam,
                    CBAM=self.CBAM))
        return layers

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder_fc(x)
        x = x.view(-1, self.in_channels_forward, self.in_height, self.in_width)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        out = self.layer4(x)
        out1 = self.iso_conv(out)
        out1=self.final_fc(out1.view(-1,360))
        return out1.view(-1,36,10)

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d)):
                nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, nonlinearity='linear')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)





In [2]:


class EnhancedMultiScaleAttentionGate(nn.Module):
    def __init__(self, input_channels, gating_channels):
        super(EnhancedMultiScaleAttentionGate, self).__init__()
        self.W_g = nn.Conv2d(gating_channels, input_channels, kernel_size=1)
        self.W_x = nn.Conv2d(input_channels, input_channels, kernel_size=1)
        self.psi = nn.Conv2d(input_channels, 1, kernel_size=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, g):
        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = self.sigmoid(self.psi(g1 + x1))
        return x * psi


class Stage2Model(nn.Module):
    def __init__(self, input_dim, out_size, features=32):
        super(Stage2Model, self).__init__()
        self.out_height, self.out_width = out_size
        
        # Embedding input dimension to spatial output
        self.input_embed = nn.Sequential(
            nn.Linear(input_dim, self.out_height * self.out_width),
            nn.LeakyReLU(0.2, inplace=True)
        )
        
        # UNet encoder-decoder architecture
        self.encoder1 = self._conv_block(2, features)
        self.pool1 = nn.Conv2d(features, features, kernel_size=4, stride=2, padding=(3, 2))

        self.encoder2 = self._conv_block(features, features * 2)
        self.pool2 = nn.Conv2d(features * 2, features * 2, kernel_size=4, stride=2, padding=(3, 2))

        self.encoder3 = self._conv_block(features * 2, features * 4)
        self.pool3 = nn.Conv2d(features * 4, features * 4, kernel_size=4, stride=2, padding=(3, 2))

        self.bottleneck = self._conv_block(features * 4, features * 8)

        self.upconv3 = nn.ConvTranspose2d(features * 8, features * 4, kernel_size=4, stride=2, padding=(3, 2))
        self.decoder3 = self._conv_block(features * 8, features * 4)

        self.upconv2 = nn.ConvTranspose2d(features * 4, features * 2, kernel_size=4, stride=2, padding=(3, 2))
        self.decoder2 = self._conv_block(features * 4, features * 2)

        self.upconv1 = nn.ConvTranspose2d(features * 2, features, kernel_size=4, stride=2, padding=(3, 2))
        self.decoder1 = self._conv_block(features * 2, 1)

        self.final_conv = nn.Conv2d(1, 1, kernel_size=1)
        self.final_fc = nn.Linear(self.out_height * self.out_width, self.out_height * self.out_width)
        
        # Attention Gates for skip connections
        self.attn_gate3 = EnhancedMultiScaleAttentionGate(features * 4, features * 4)
        self.attn_gate2 = EnhancedMultiScaleAttentionGate(features * 2, features * 2)
        self.attn_gate1 = EnhancedMultiScaleAttentionGate(features, features)
        
        self._initialize_weights()

    def forward(self, x, output1):
        # Embed the input and concatenate with Stage1 output
        input_embed = self.input_embed(x)
        input_embed = input_embed.view(-1, 1, self.out_height, self.out_width)
        stage2_input = torch.cat([input_embed, output1.unsqueeze(1)], dim=1)

        # UNet forward pass
        enc1 = self.encoder1(stage2_input)
        enc2 = self.encoder2(self.pool1(enc1))
        enc3 = self.encoder3(self.pool2(enc2))

        bottleneck = self.bottleneck(self.pool3(enc3))

        # Decoder with attention gates
        dec3 = self.upconv3(bottleneck)
        attn3 = self.attn_gate3(enc3, dec3)  # Apply attention gate to skip connection
        dec3 = torch.cat([dec3, attn3], dim=1)
        dec3 = self.decoder3(dec3)

        dec2 = self.upconv2(dec3)
        attn2 = self.attn_gate2(enc2, dec2)  # Apply attention gate to skip connection
        dec2 = torch.cat([dec2, attn2], dim=1)
        dec2 = self.decoder2(dec2)

        dec1 = self.upconv1(dec2)
        attn1 = self.attn_gate1(enc1, dec1)  # Apply attention gate to skip connection
        dec1 = torch.cat([dec1, attn1], dim=1)
        dec1 = self.decoder1(dec1)
        
        output2 = self.final_conv(dec1)
        output2 = self.final_fc(output2.view(-1, self.out_height * self.out_width))
        
        return output2.view(-1, self.out_height, self.out_width)

    def _conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2, inplace=True)
        )

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d)):
                nn.init.kaiming_normal_(m.weight, nonlinearity='leaky_relu', a=0.2) 
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, nonlinearity='linear')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)


In [3]:

# Training function for Stage 2 with loss history, test loss, and batch size scheduler
def train_stage2(stage1_model, stage2_model, train_dataset, test_loader, optimizer, scheduler, criterion, num_epochs, scaler, batch_epoch, batches):
    stage1_model.eval()  # Stage1Model is frozen and in evaluation mode
    stage2_model.train()  # Stage2Model will be trained
    
    train_loss_history = []  # List to store training loss per epoch
    test_loss_history = []   # List to store test loss per epoch
    
    # Initialize batch size scheduler
    current_batch_idx = 0
    batch_size = batches[current_batch_idx]
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    # TQDM progress bar for epochs
    with tqdm(range(1, num_epochs + 1), desc="Training Stage 2", unit="epoch") as progress_bar:
        for epoch in progress_bar:
            running_train_loss = 0.0  # Track running train loss
            
            # Update batch size based on the current epoch
            if  epoch == batch_epoch[current_batch_idx]:
                current_batch_idx += 1
                batch_size = batches[current_batch_idx]
                train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

            # Loop over batches in training set
            for inputs, iso_targets, res_targets in train_loader:
                optimizer.zero_grad()  # Reset gradients

                # Forward pass through Stage 1 (with no gradient tracking)
                with torch.no_grad():
                    out1 = stage1_model(inputs)
                
                # Forward pass through Stage 2 (with mixed precision)
                with autocast('cuda'):
                    outputs = stage2_model(inputs, out1)  # Pass Stage1 output to Stage2
                    loss = criterion(outputs, res_targets)  # Compute loss
                
                # Backward pass and optimization
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()

                running_train_loss += loss.item() * inputs.size(0)  # Update running loss
            
            # Calculate average training loss for the epoch
            epoch_train_loss = running_train_loss / len(train_loader.dataset)
            train_loss_history.append(epoch_train_loss)
            
            # Evaluate test loss after each epoch
            stage2_model.eval()  # Switch to evaluation mode for test set
            running_test_loss = 0.0
            with torch.no_grad():
                for inputs, iso_targets, res_targets in test_loader:
                    with autocast('cuda'):
                        out1 = stage1_model(inputs)
                        outputs = stage2_model(inputs, out1)
                        loss = criterion(outputs, res_targets)
                    running_test_loss += loss.item() * inputs.size(0)
            
            epoch_test_loss = running_test_loss / len(test_loader.dataset)
            test_loss_history.append(epoch_test_loss)
            
            # Switch back to training mode
            stage2_model.train()
            
            # Update the learning rate scheduler based on the loss
            scheduler.step(epoch_train_loss)
            
            # Update the progress bar with current epoch and training loss
            progress_bar.set_postfix({
                'Epoch': epoch,
                'Train Loss': f"{epoch_train_loss:.4f}",
                'Test Loss': f"{epoch_test_loss:.4f}",
                'batch size': f"{batch_size:.0f}",
                'Lr ': f"{optimizer.param_groups[0]['lr']:.8f}"
            })
    
    return train_loss_history, test_loss_history



In [ ]:

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Set the seed
set_seed(7)

# Instantiate Stage1Model
conv_block = Resnext_bottleneck_block
convt_block = Resnext_bottleneck_transpose_block
cbam_module = CBAM

stage1_params = {
    'decoder_in': (512, 4, 2),
    'out_size': (36, 10),
    'conv_block': conv_block,
    'convt_block': convt_block,
    'n_blocks': (2, 3, 2, 4),
    'decoder_channel_shrink': 1,
    'width_ratio': 1,
    'use_cbam': (True, False, True, False),
    'CBAM': cbam_module,
    'CBAM_skip': True,
}

stage1_model = Stage1Model(**stage1_params).to(device)



# Instantiate Stage2Model with UNet inside
stage2_model = Stage2Model(input_dim=7, out_size=(36, 10), ).to(device)




In [6]:
# Define optimizer, scheduler, and loss function for Stage 2
learning_rate_stage2 = 1e-3
num_epochs_stage2 = 600
weight_decay_stage2 = 1e-5

# torch.use_deterministic_algorithms(False) 
    
optimizer_stage2 = optim.Adam(stage2_model.parameters(), lr=learning_rate_stage2, weight_decay=weight_decay_stage2)
scheduler_stage2 = ReduceLROnPlateau(optimizer_stage2, mode='min', factor=0.5, patience=15)
criterion_stage2 = nn.L1Loss()

# Data loaders for Stage 2
# batch_size_stage2 = 16
# train_loader_stage2 = DataLoader(train_dataset, batch_size=batch_size_stage2, shuffle=True)
test_loader_stage2 = DataLoader(test_dataset, batch_size=512, shuffle=False)

# Mixed precision setup for Stage 2 (fix for future warning)
scaler_stage2 = torch.amp.GradScaler()

batch_epoch = [200, 400, 1000]  
batches = [32, 64, 256]                      

# Example usage for Stage 2
train_loss_history_stage2, test_loss_history_stage2 = train_stage2(
    stage1_model, stage2_model, train_dataset, test_loader_stage2, 
    optimizer_stage2, scheduler_stage2, criterion_stage2, 
    num_epochs_stage2, scaler_stage2, batch_epoch, batches
)

Training Stage 2: 100%|██████████| 600/600 [18:39<00:00,  1.87s/epoch, Epoch=600, Train Loss=0.0016, Test Loss=0.0037, batch size=256, Lr =0.00006250]


In [9]:
import torch
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr
# Function to set the random seed for reproducibility
def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True)

seed = 42
set_seed(seed)

# Function to retrieve model outputs for both training and testing data
def get_model_outputs(model_stage1, model_stage2, device, X_train_t, X_test_t):
    model_stage1.to(device)
    model_stage2.to(device)
    model_stage1.eval()
    model_stage2.eval()

    with torch.no_grad():
        X_train_t = X_train_t.to(device)
        X_test_t = X_test_t.to(device)

        # Stage 1 forward pass
        train_output1 = model_stage1(X_train_t)
        test_output1 = model_stage1(X_test_t)

        # Stage 2 forward pass
        train_output2 = model_stage2(X_train_t, train_output1)
        test_output2 = model_stage2(X_test_t, test_output1)

    return (train_output1, train_output2), (test_output1, test_output2)


# Function to calculate metrics for true vs. predicted values
# Function to calculate metrics for true vs. predicted values
def calculate_metrics(true, pred):
    # Move tensors to CPU and convert to NumPy arrays if they are not already
    if isinstance(true, torch.Tensor):
        true = true.cpu().numpy()
    if isinstance(pred, torch.Tensor):
        pred = pred.cpu().numpy()

    # Flatten the arrays
    true_flat = true.reshape(-1)
    pred_flat = pred.reshape(-1)

    # Calculate metrics
    mse = mean_squared_error(true_flat, pred_flat)
    mae = mean_absolute_error(true_flat, pred_flat)
    r2 = r2_score(true_flat, pred_flat)
    mape = np.mean(np.abs((true_flat - pred_flat) / true_flat)) * 100
    r, _= pearsonr(true_flat, pred_flat)
    return mse, mae, r2, mape, r

# Retrieve model outputs on CUDA (if available) or CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
(train_output1, train_output2), (test_output1, test_output2) = get_model_outputs(stage1_model, stage2_model, device, X_train_t, X_test_t)

# Compute predictions for training and testing data
train_pred1 = train_output1
train_pred2 = (train_output2)
test_pred1 = (test_output1)
test_pred2 = (test_output2)

# Use the wrapper function for true values; no need to move to device since they're already NumPy arrays
train_org1 =iso_train_t
train_org2 = res_train_t
test_org1 = iso_test_t
test_org2 = res_test_t

# Calculate metrics for both outputs
train_metrics1 = calculate_metrics(train_org1, train_pred1)
test_metrics1 = calculate_metrics(test_org1, test_pred1)
train_metrics2 = calculate_metrics(train_org2, train_pred2)
test_metrics2 = calculate_metrics(test_org2, test_pred2)

# Print results for training data (Output 1)
print(f"Train Output 1 - MSE: {train_metrics1[0]}, MAE: {train_metrics1[1]}, R2: {train_metrics1[2]}, MAPE: {train_metrics1[3]}, R: {train_metrics1[4]}")
# Print results for training data (Output 2)
print(f"Train Output 2 - MSE: {train_metrics2[0]}, MAE: {train_metrics2[1]}, R2: {train_metrics2[2]}, MAPE: {train_metrics2[3]}, R: {train_metrics2[4]}")

# Print results for testing data (Output 1)
print(f"Test Output 1 - MSE: {test_metrics1[0]}, MAE: {test_metrics1[1]}, R2: {test_metrics1[2]}, MAPE: {test_metrics1[3]}, R: {test_metrics1[4]}")
# Print results for testing data (Output 2)
print(f"Test Output 2 - MSE: {test_metrics2[0]}, MAE: {test_metrics2[1]}, R2: {test_metrics2[2]}, MAPE: {test_metrics2[3]}, R: {test_metrics2[4]}")


Train Output 1 - MSE: 0.0001483383384766057, MAE: 0.0053728362545371056, R2: 0.9992623329162598, MAPE: 0.8887980133295059, R: 0.9996311442561985
Train Output 2 - MSE: 1.594929381099064e-05, MAE: 0.001505478867329657, R2: 0.980930745601654, MAPE: 129.27244901657104, R: 0.9904264102134283
Test Output 1 - MSE: 0.000298578612273559, MAE: 0.008552854880690575, R2: 0.9985069632530212, MAPE: 1.3339451514184475, R: 0.9992534057816923
Test Output 2 - MSE: 0.00015137006994336843, MAE: 0.003702885238453746, R2: 0.8158720135688782, MAPE: 224.90262985229492, R: 0.9035423793659445
